# Modelling patch-clamp experiments: compensation

In the [last notebook](./artefacts-1-modelling-patch-clamp.ipynb), we introduced an electrical schematic that can serve as a model of the patch-clamp set up, with distortions by leak, offsets, and unwanted capacitances.
To compensate for these unwanted effects, patch-clamp amplifiers contain special circuitry.
In this notebook we extend our model to include the effects of these compensations, as modelled in [Lei et al., 2020](https://doi.org/10.1098/rsta.2019.0348).

We will deal mostly with _transient_ distortions of the recorded output signal, which we call _artefacts_, and with transient differences between the true and intended membrane potential, which are an example of _imperfect control_.
These terms, and general strategies for dealing with their effects, are discussed in [Appendix H](./appendix-H-strategies.ipynb).
Stochastic and periodic noise are not discussed here, but a brief discussion is given in [Appendix I](./appendix-I-inspecting-noise.ipynb).

## Zeroing and the liquid junction potential ($E_\text{off}$)

In the last notebook we introduced $E_\text{off}$, which represents the sum of various voltage offsets that are incurred in the experimental setup.
This offset is usually corrected by a dial or a digital control that allows the experimenter to subtract an equal offset $-E_\text{off}^*$.
To allow for the idea that this correction is imperfect, either because it was set imperfectly or because $E_\text{off}$ drifted over time, we now introduce a symbol for the error in the offset correction $E_\text{off}^\dagger$ as
\begin{equation}
E_\text{off}^\dagger = E_\text{off} - E_\text{off}^*
\end{equation}

<img src="resources/patch-comp-1-Eoff.png" style="margin:auto" />

_**Figure 1**: The voltage offset has been corrected until only a small error $E_\text{off}^\dagger$ remains._

### The liquid junction potential

A liquid junction potential (LJP) arises wherever two liquids containing different concentrations of ions are in contact.
In manual patch clamp this is the case just before the experiment, when the pipette is in the bath but not yet attached to the cell.
The correction $E_\text{off}*$ is usually determined just before the final approach to the cell, and so includes this LJP.
Once a connection to the cell is made, the pipette fluid is in contact with the cytosol and because both fluids are similar an LJP no longer exists.
In other words, the LJP is removed from $E_\text{off}$, but not from $E_\text{off}^*$.

The correction for this overcorrection is called LJP correction, and proceeds as follows:

1. The LJP is calculated by entering the bath and pipette solutions into an LJP calculating program.
2. We can then adjust all $V_\text{cmd}$ values to obtain the desired $V_m$ (*a priori* correction) or simply accept that $V_m$ is shifted from the desired value and account for this in our analysis (*a posteriori* correction). 

Some systems allow the LJP to be entered into the recording software, so that *a priori* correction can be performed without further user input.

A detailed description of the correction procedure is provided in [Appendix J](./appendix-J-liquid-junction-potential.ipynb).
In short, if you have to do manual *a posteriori* correction then you **subtract** the LJP from the applied or measured voltage to get the true value.

### We won't explicitly treat the LJP

In the remainder of this notebook we will assume LJP correction has been performed by adjusting $V_\text{cmd}$.
We will treat $E_\text{off}^\dagger$ as a small number that does not include the LJP.

## Fast capacitance correction ($C_p$)

The next compensation circuitry we include attempts to charge the capacitor $C_p$ without affecting the current through the cell.
To do this, a prediction of the current flowing into $C_p$ is made, and "injected" into the circuitry.

<img src="resources/patch-comp-2-Cp.png" style="margin:auto" />

_**Figure 2**: Fast capacitance correction aims to charge $C_p$ without affecting the current through the cell._

In the diagram above, we have drawn an active component that somehow generates the current $I_\text{inj}$, but does not draw any current from the node at $V_c$.
As an equation for $I_\text{inj}$ we'll use

\begin{equation}
I_\text{inj} = C_p^* \dot{V}_c
\end{equation}

where $C_p^*$ is the _estimated_ parasitic capacitance.
(In a real implementation a fixed $C$ is used, but an amplifier with a variable gain $A$ is attached, so that $A \cdot C_\text{fixed}$ can be made to match $C_p$.)

Note that the above equation is problematic for voltage step protocols, in which $\dot{V}_c$ is either $0$ or $\pm\infty$.
This is less of a problem in practice, as stray capacitances and other factors will conspire to "round" $V_\text{cmd}$ a little and make $\dot{V}_\text{cmd}$ finite.
However, it does still make matching $C_p$ and $C_p^*$ very difficult in practice, and [Sigworth 1995a](https://doi.org/10.1007/978-1-4419-1229-9_4) proposes a schematic in which

1. $V_\text{cmd}$ is passed through a 10 $\mu$s low-pass filter.
2. A split is made, and the branch connected to the op amp is filtered with a further 0.5 $\mu$s.
3. The branch used to create $I_\text{inj}$ is filtered with a variable time constant between 0 and 1 $\mu$s, which is manually calibrated to obtain a good step response.

This means that, even with _perfect_ capacitance correction, we should still expect a rounding off of any voltage step protocols!

In this notebook we will assume the rounding is well modelled by other capacitances in the system, and proceed with the equation given above.

## Slow capacitance correction ($C_m$)

Next, we add a similar correction term for the much larger capacitance $C_m$.
As with $C_p$ correction, there are practical considerations that complicate real-world implementations ([Sigworth 1995a](https://doi.org/10.1007/978-1-4419-1229-9_4)) but for our model we can simply add another term based on the estimated membrane capacitance $C_m^*$:

\begin{equation}
I_\text{inj} = C_p^* \dot{V}_c + C_m^* \dot{V}_c
\end{equation}

## What if we don't correct?

Since we're going to be modelling both physical effects (e.g. fast and slow capacitative transients) and their (imperfect) corrections, it makes sense to ask why we're correcting at all.
Surely it would be simpler to switch off the corrections and model only the physical effects?

Unfortunately there are two practical difficulties with this approach (see also [Sigworth 1995a](https://doi.org/10.1007/978-1-4419-1229-9_4)).
Firstly, the large charging currents can exceed the limits of the A/D converter used to digitise the output voltage.
For the samples during which this _clipping_ occurs, we record only the maximum (or minimum) value, so information about the current is lost (making fitting harder).
Amplifiers in the A/D converter may also go into _saturation_, which prolongs the clipping until the amplifiers have recovered.

Secondly, large transient currents can cause the op amp in the feedback circuit to saturate, leading to a loss of control over the membrane voltage until the amplifier has recovered.


## Series resistance compensation ($R_s$)

Slow capacitance correction can hide the membrane charging current from our recordings, but the membrane still needs to be charged, and in whole-cell patch clamp this is a relatively slow process, with a time constant given by $R_s C_m$.
The milliseconds it takes for the membrane to reach the desired potential can have a pronounced effect on fast ion currents such as the fast sodium current INa, and many examples of this can be found in the literature (e.g. [Sherman et al., 1999](https://doi.org/10.1016/S0006-3495(99)77093-1), [Clerx et al., 2015](http://michaelclerx.com/publications/files/clerx-collins-volders-2015-applied-estimation-pre-print.pdf), [Montnach et al., 2021](https://doi.org/10.1038/s41598-021-82077-8), [Abrasheva et al. (2023)](https://doi.org/10.1101/2023.06.06.543894)).

Several strategies exist to shorten the membrane charging time.
One of the more straightforward ones, described in [Hodgkin et al. 1952](https://physoc.onlinelibrary.wiley.com/doi/10.1113/jphysiol.1952.sp004716) and [Sigworth 1995a](https://doi.org/10.1007/978-1-4419-1229-9_4) and used in [Lei et al., 2020](https://doi.org/10.1098/rsta.2019.0348), is to _feed a fraction of $V_\text{out}$ forward into $V_c$_.
In the schematic, we'll show this with two new components: an active component labelled _R_ that generates the voltage to be added, and a _summing amplifier_, labelled $\Sigma$:

<img src="resources/patch-comp-3-Rs.png" style="margin:auto" />

_**Figure 3**: Series resistance compensation feeds a fraction of $V_\text{out}$ into $V_\text{ref}$ via a summing amplifier._

In this new set-up

\begin{align}
V_\text{out} = V_o - V_\text{ref}
\end{align}

And so the extra voltage added by the series resistance compensation will be zero when $V_o = V_\text{ref}$.

### Feed-forward speeds up charging

To see why the feed-forward mechanism makes sense, we first look at the situation where there is no ionic current ($I = 0$), no leak current ($I_\text{leak} = 0$) and a perfectly compensated voltage offset ($E_\text{off}^\dagger = 0$).
Under these conditions, the only currents in the system are the charging currents, most notably $C_m\dot{V}_m$.
These are only active while the membrane is imperfectly clamped ($V_m \neq V_\text{ref}$), and so _exaggerating_ the difference between $V_\text{ref}$ and $V_m$ (or $V_p$) should speed up the charging process.
When the membrane is perfectly clamped the capcitative currents are all zero so that $V_m = V_p = V_\text{ref} = V_o$, $V_\text{out} = 0$, and the series resistance compensation term is zero.
We can also recognise that the response to any overshoot is going to be similarly exaggerated, so this kind of feed-forward comes with a risk of increased instability.

### ...and reduces steady-state voltage error

Next, we consider the steady-state case but with a constant ionic current (or leak current).
A simplified schematic for this case is shown below.

<img src="resources/patch-comp-3b-Rs.png" style="margin:auto" />

_**Figure 4**: A simplified schematic for the steady-state with series resistance compensation._

Using $\beta$ for the fraction of $V_\text{out}$ fed into $V_\text{ref}$ we find

\begin{align}
V_\text{out} = V_o - V_\text{ref}
             = (V_p + IR_f) - V_\text{ref}
             = IR_f
\end{align}
and
\begin{align}
V_\text{ref} = V_c + \beta V_\text{out} = V_c + \beta IR_f
\end{align}
for
\begin{align}
V_m = V_p - R_s I = V_c + (\beta R_f - R_s) I
\end{align}

so if we can choose $\beta R_f = R_s$ we can compensate for the voltage drop over $R_s$ entirely.
In our model, we will assume that the feed-forward rate is set based on an estimate of the series resistance $R_s^*$ and a _fractional compensation_ rate $\alpha$:

\begin{align}
V_\text{ref} = V_c + \alpha R_s^* I
\end{align}

In practice, the feed-forward system is a bit more complicated, and the whole system once again acts as an oscillator with a damping factor dependent on $\alpha$.
To avoid "ringing", $\alpha$ is usually limited to about 70 or 80%.

### The summing amplifier has a finite speed

Finally, we follow [Lei et al., 2020](https://doi.org/10.1098/rsta.2019.0348) in incorporating the finite speed of the summing amplifier as

\begin{align}
\dot{V}_\text{ref} = \frac{V_c + \alpha R_s^*I - V_\text{ref}}{\tau_\text{sum}}
\end{align}

where $\tau_\text{sum} \approx 0.04$ ms.

## Filtering the output

Filter 1

## Simulations

## Conclusion